# Just playing around here

Note to self: Make sure to run this notebook in the conda environment to have packages available. 

    conda activate
    jupyter notebook

##### Coolio, let's start by listing instances. 

In [21]:
!aws ec2 describe-instances \
    --query 'Reservations[*].Instances[*].{Type:InstanceType,\
                                           LaunchTime:LaunchTime,\
                                           Instance:InstanceId,\
                                           KeyName:KeyName,\
                                           State:State.Name,\
                                           Name:Tags[?Key==`cluster-name`]|[0].Value}' \
    --output table

----------------------------------------------------------------------------------------------------------------------------------
|                                                        DescribeInstances                                                       |
+---------------------+---------------------+---------------------------+------------------------+-----------------+-------------+
|      Instance       |       KeyName       |        LaunchTime         |         Name           |      State      |    Type     |
+---------------------+---------------------+---------------------------+------------------------+-----------------+-------------+
|  i-06db20077a6e44cad|  PSADSC291KeyPair   |  2020-04-19T04:15:15.000Z |  dsc291hw2arndt        |  shutting-down  |  t2.micro   |
|  i-039be9e81693344c6|  PSADSC291KeyPair   |  2020-04-19T08:28:57.000Z |  dsc291hw2anothertest2 |  terminated     |  t2.micro   |
|  i-0c71d929d331d1a9d|  SGabaDSC291KeyPair |  2020-04-16T18:44:26.000Z |  sgaba-te

### a few options for shutting down instances 

In [ ]:
# !aws ec2 stop-instances --instance-ids "i-0105aaca12dbcb0e1" "i-0f36f5a941f2fee32"
# !aws ec2 stop-instances --instance-ids "i-0c71d929d331d1a9d" --hibernate
!aws ec2 terminate-instances --instance-ids "i-06db20077a6e44cad"


### Create an ec2 instance using aws-jupyter package

In [3]:
# !aws-jupyter create -c 1 --name dsc291hw2psa --region us-west-2 --type t2.micro
# !aws-jupyter check

In [4]:
# just to make sure things work
!pwd 
# !aws-jupyter access  # prints credentials!!

# enable inline graphics
%pylab inline  

/home/parndt/DSC291/Public-DSC291/hw2
Populating the interactive namespace from numpy and matplotlib


## Python wrapper for aws-jupyter

We could put this in a separate file (as demonstrated in class), but here I wanna understand what's going on...

In [5]:
import subprocess
from time import sleep
import argparse

def run_command(command,debug=False):
    if debug:
        print('running ',command)
    p=subprocess.Popen(command.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out=p.communicate()
    stdout=out[0].decode()
    stderr=out[1].decode()
    outputs={"stderr":stderr,
             "stdout":stdout}
    if debug:
        print(outputs)
    return outputs

In [6]:
class aws_jupyter:
    """
    A python wrapper around the script **aws-jupyter**
    """
    
    def check(self):
        """
        Check on the status of the ec2 instance
        Returns status
        -------
        0 = No instance running under this name 
        
        1 = instance in the process of starting up
        
        2 = instance available        
        
        -1 = Status could not be parsed
        """
        check_cmd = "aws-jupyter check --name %s"%self.name
        self.decoded=run_command(check_cmd)
        stdout=self.decoded['stdout']
        if('The Jupyter Notebook is running on the cluster at the address below.' in stdout):
            print(stdout)
            return 2
        elif("No instance found in the cluster" in stdout):
            return 0
        elif("Cluster is not ready. Please check again later." in stdout):
            return 1
        else:
            print(" ...did not recognize check status")
            # print(stdout)
            return -1

    def run(self,scriptname,files=[],credentials="",waitforoutput=True,printoutput=False): 
        """
        Run a local script on the remote instance
        Parameters
        ----------
        scriptname : TYPE
            DESCRIPTION.
        files : TYPE, optional
            DESCRIPTION. The default is [].
        credentials : TYPE
            DESCRIPTION.
        waitforoutput : TYPE, optional
            DESCRIPTION. The default is True.
        Returns
        -------
        None.
        """
            
        running_cmd = "aws-jupyter run -s {script}"
        runcmd = running_cmd.format(script=scriptname)
        if len(files)>0:
            runcmd += " --files"
            for i in range(len(files)):
                runcmd += " {fn}".format(fn=files[i])
        if waitforoutput:
            runcmd += " --output"
        print("running command: {runcmd}".format(runcmd=runcmd))
        out = run_command(runcmd,debug=False)
        if printoutput:
            print(out['stdout'])
            print(out['stderr'])

    def retrieve():
        """ call aws-jupyter retrieve"""
        return
    
    def terminate(self,printoutput=False):
        """ call aws-jupyter terminate"""
        term_cmd = "yes | aws-jupyter terminate --name %s" % self.name
        print("terminate instance: {term_cmd}".format(term_cmd=term_cmd))
        out = run_command(term_cmd)
        if printoutput:
            print(out['stdout'])
            print(out['stderr'])
            
    def __init__(self,name='instance',count=1,_type='t3.large',spot=0):
        """
        Create a cluster of instances on ec2
        Parameters
        ----------
        name : TYPE, optional
            DESCRIPTION. The default is 'instance'.
        count : TYPE, optional
            DESCRIPTION. The default is 1.
        _type : TYPE, optional
            DESCRIPTION. The default is 't3.large'.
        spot : TYPE, optional
            DESCRIPTION. The default is 0.
        Returns
        -------
        None.
        """
        self.name=name
        status=self.check()
        if status==1:
            print("cluster %s not ready yet"%self.name)
            return
        elif status==2:
            print("cluster running")
            return

        create_cmd = "aws-jupyter create -c {count} --name {name} --type {_type}"
        command=create_cmd.format(count=count,name=name,_type=_type)
        if spot>0:
            command.append(" --spot %4.2f"%spot)
        out=run_command(command)
        print("initiated instance:",command)
 
        i=0; wait_period=10
        while True:
            status=self.check()
            if status==2:
                # print(stdout)
                break
            print('\r check',i*wait_period,end='')
            i+=1
            sleep(wait_period)

### Coming up with a dumb script to run on each instance

---> this is where we probably wanna do something more interesting haha

In [11]:
program_to_run = '''#!/usr/bin/env python3

import numpy as np
from time import time
from os.path import isfile,isdir
from os import mkdir,chdir,getcwd
import pickle as pk

times = []
nTrials = 1000
nFibonacci = 10**4

# a dumb random computational task, on which to compare instances
inst_starttime = time()
for k in range(nTrials):
    starttime = time()
    fibonacci_previous = 1
    fibonacci_current = 1
    for i in range(2,nFibonacci):
        fibonacci_next = fibonacci_previous + fibonacci_current
        fibonacci_previous = fibonacci_current
        fibonacci_current = fibonacci_next
    DT = time() - starttime
    times.append(DT)
print(time() - inst_starttime)
    
#hist(times,bins=50)
exec_dir = getcwd()
log_dir = exec_dir + '/measurement_logs/'
if not isdir(log_dir):
    mkdir(log_dir)
fn = log_dir + 'stats.pkl'
with open(fn,'wb') as times_pkl:
    pk.dump(times,times_pkl,protocol=pk.HIGHEST_PROTOCOL)'''

theFile = open('ec2test.py','w') 
theFile.write(program_to_run) 
theFile.close() 

In [ ]:
!cat ec2test.py

### Create an EC2 instance thorugh aws-jupyter
using t2.micro here for now to test things out, because those are free up to 250hrs/month...

In [ ]:
thisType = "t2.micro"
thisName = "dsc291hw2" + thisType.replace('.','')
thisInst = aws_jupyter(name="dsc291hw2anothertest2", count=1, _type=thisType)
# thisInst = aws_jupyter(name="dsc291hw2arndt3", count=1, _type="m3.xlarge")

### Now run the startup script
(still having trouble with the --files argument: when left empty it will use the last used value from the config file, and I can't figure out how to tell aws-jupyter that there's NO input files at all... As a temporary fix, I edited the config file manually.)

In [ ]:
thisInst.run(scriptname='startup.sh',printoutput=True)

### and now the test script


In [ ]:
thisInst.run(scriptname='ec2test.py',printoutput=True)

### Retrieve results
...and rename file by instance type!

In [ ]:
!mkdir from_remote
!aws-jupyter retrieve --remote /home/ubuntu/workspace/measurement_logs/stats.pkl --local from_remote/
new_fn = 'from_remote/stats_' + thisType.replace('.','-') + '.pkl'
!mv from_remote/worker-0/stats.pkl $new_fn

### Terminate the instance
Can't get it to work through the python object here, just call aws-jupyter directly...

In [ ]:
# thisInst.terminate(printoutput=True)  # <--- this doesn't work!
!yes | aws-jupyter terminate --name $thisInst.name

In [ ]:
# To view this notebook as slides
!jupyter nbconvert hw2psa.ipynb --to slides --post serve